# CMSD Data Analyst Skills Assessment
In this notebook, I will outline my overall data preparation process. This environment is setup using Python, however, all of my queries will use SQL and the data is stored in a SQLite db on my computer.

I have the steps below, however, my general process was to check through the data (this was also done in Excel, and later in Tableau), then form Unions from each file type. I found this process to be a bit more straightforward since I was able to immediately check whether I had the data I wanted from each of the three main spreadsheet types I used from the Ohio State Report Card site.

Once the unions were completed, I joined all of the tables into one final output table, and then moved onto working in Tableau.

## General Steps
1) Download and verify files from https://reportcard.education.ohio.gov/download
2) Convert files to .csv format and create tables in SQLite (I used DB Browser for SQLite for this step)
3) Create Jupyter SQL environment and connect to db
4) Filter data based on specified requirements - School Years and District IRN (043786)
5) Final query csv ouput file with 6 required columns

In [210]:
%%capture
!pip install ipython-sql
!pip install sqlalchemy

In [23]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %sql  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%py

In [1]:
%%capture
%load_ext sql
# Establishing SQL environment and connecting to SQLite db
import sqlalchemy
sqlalchemy.create_engine("sqlite:///CMSD_db.db")
%sql sqlite:///CMSD_db.db  

In [2]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

## Achievement Building

In this section, I am going to work on the three tables for Building Achievement.

The general process will involve filtering by District IRN, removing unnecessary columns, and inserting a year column. Due to the nature of this project, I opted for using temporary tables since this was completed in one session, although this is achievable with views as well.

Overall, I worked on each category separately to ensure my naming conventions were consistent, and that the data was comparable year to year. 

### Creating Temp Table for Achievement Building School Year 2015-2016

In [196]:
%%sql
SELECT *
FROM Achievement_Building_1516
WHERE DistrictIRN = '043786'
LIMIT 3

/*Just a basic query to check everything is working as intended before making temporary tables

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,County,Region,Address,CityandZipCode,Phone#,Principal,PerformanceIndexScore2015-16,PerformanceIndexPercent2015-16,LetterGradeofPerformanceIndex,PercentofStudentsNotTested,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,GiftedPerformanceIndexScore2015-16,GiftedPerformanceIndex2015-16,PercentofGiftedStudentsNotTested,PercentofGiftedStudentsBelow,PercentofGiftedStudentsBasic,PercentofGiftedStudentsProficient,PercentofGiftedStudentsAccelerated,PercentofGiftedStudentsAdvanced,PercentofGiftedStudentsAdvancedPlus,PerformanceIndexScore2014-15,PerformanceIndexScore2013-14,Watermark
224,Adlai Stevenson School,43786,Cleveland Municipal,Cuyahoga,Region 3,18300 Woda Avenue,"Cleveland, OH, 44122-6441",(216) 482-2950,Christopher T. Wyland,43.903,36.6,F,0,69.9,19,7.6,2.5,1,0,NC,NC,NC,NC,NC,NC,NC,NC,NC,55.307,68.009,None
489,Almira,43786,Cleveland Municipal,Cuyahoga,Region 3,3375 W 99th St,"Cleveland, OH, 44102-4642",(216) 838-6150,Laverne Hooks,46.384,38.7,F,0,66.2,19.2,10.9,3.2,0.5,0,NC,NC,NC,NC,NC,NC,NC,NC,NC,59.407,67.316,None
729,Andrew J Rickoff,43786,Cleveland Municipal,Cuyahoga,Region 3,3500 E 147th St,"Cleveland, OH, 44120-4834",(216) 838-4150,Gloriane R. Smith,44.327,36.9,F,0.3,68.5,19.4,8.7,2.4,0.7,0,NC,NC,NC,NC,NC,NC,NC,NC,NC,51.929,62.727,None


In [24]:
%%sql
CREATE TEMPORARY TABLE Temp_Achievement_Building_1516 AS
SELECT "BuildingIRN", "BuildingName", "DistrictIRN", "DistrictName", "PerformanceIndexScore2015-16" AS "PerformanceIndexScore", "PerformanceIndexPercent2015-16" AS "PerformanceIndexPercent", "LetterGradeofPerformanceIndex", "PercentofStudentsBelow", "PercentofStudentsBasic", "PercentofStudentsProficient", "PercentofStudentsAccelerated", "PercentofStudentsAdvanced", "PercentofStudentsAdvancedPlus", '2015-2016' AS SchoolYear
FROM Achievement_Building_1516
WHERE "DistrictIRN" = '043786';
/* In this temp table, I isolated the columns I wanted to keep and added a school year column for organization.
Additionally, I some extra columns for checking and possible analysis to explore later on.
Lastly, I used quotations around all of the column names for consistency due to irregular naming.

 * sqlite:///CMSD_db.db
Done.
0 rows affected.


[]

In [56]:
%%sql
DROP TABLE IF EXISTS Temp_Building_Ratings_1617

 * sqlite:///CMSD_db.db
Done.


[]

In [25]:
%%sql
PRAGMA table_info(Temp_Achievement_Building_1516)
/* This Temp Table has everything I intended, additional columns might be added later, 
but it looks pretty good from here.

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,BuildingIRN,INT,0,None,0
1,BuildingName,TEXT,0,None,0
2,DistrictIRN,INT,0,None,0
3,DistrictName,TEXT,0,None,0
4,PerformanceIndexScore,TEXT,0,None,0
5,PerformanceIndexPercent,TEXT,0,None,0
6,LetterGradeofPerformanceIndex,TEXT,0,None,0
7,PercentofStudentsBelow,TEXT,0,None,0
8,PercentofStudentsBasic,TEXT,0,None,0
9,PercentofStudentsProficient,TEXT,0,None,0


In [211]:
%%sql
SELECT *
FROM Temp_Achievement_Building_1516
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,PerformanceIndexScore,PerformanceIndexPercent,LetterGradeofPerformanceIndex,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,SchoolYear
224,Adlai Stevenson School,43786,Cleveland Municipal,43.903,36.6,F,69.9,19,7.6,2.5,1,0,2015-2016


In [35]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Achievement_Building_1516
/* Checking to make sure I have the desired results for DistrictIRN and DistrictName. I included DistrictName as 
a second layer/additional means of checking that I only had Cleveland Schools. At this point, due to the low
record count, I also double checked that this was the correct result in Excel to confirm before moving forward.

 * sqlite:///CMSD_db.db
Done.


building,District,Name
101,1,1


### Creating Temp Table for Achievement Building School Year 2016-2017

In [212]:
%%sql
SELECT *
FROM Achievement_Building_1617
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,County,Region,Address,CityandZipCode,Phone#,Principal,PerformanceIndexScore2016-17,PerformanceIndexPercent2016-17,LetterGradeofPerformanceIndex,PercentofStudentsNotTested,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,GiftedPerformanceIndexScore2016-17,GiftedPerformanceIndex2016-17,PercentofGiftedStudentsNotTested,PercentofGiftedStudentsBelow,PercentofGiftedStudentsBasic,PercentofGiftedStudentsProficient,PercentofGiftedStudentsAccelerated,PercentofGiftedStudentsAdvanced,PercentofGiftedStudentsAdvancedPlus,PerformanceIndexScore2015-16,PerformanceIndexScore2014-15,Watermark
224,Adlai Stevenson School,43786,Cleveland Municipal,Cuyahoga,Region 3,18300 Woda Avenue,"Cleveland, OH, 44122-6441",(216) 838-5300,Christopher T. Wyland,49.271,41.1,F,0.2,60.4,22.5,10.8,5.4,0.8,0.0,NC,NC,0.0,33.3,11.1,11.1,33.3,11.1,0.0,43.903,55.307,None


In [27]:
%%sql
CREATE TEMPORARY TABLE Temp_Achievement_Building_1617 AS
SELECT "BuildingIRN", "BuildingName", "DistrictIRN", "DistrictName", "PerformanceIndexScore2016-17" AS "PerformanceIndexScore", "PerformanceIndexPercent2016-17" AS "PerformanceIndexPercent", "LetterGradeofPerformanceIndex", "PercentofStudentsBelow", "PercentofStudentsBasic", "PercentofStudentsProficient", "PercentofStudentsAccelerated", "PercentofStudentsAdvanced", "PercentofStudentsAdvancedPlus", '2016-2017' AS SchoolYear
FROM Achievement_Building_1617
WHERE "DistrictIRN" = '043786';

 * sqlite:///CMSD_db.db
Done.


[]

In [38]:
%%sql
PRAGMA table_info(Temp_Achievement_Building_1617)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,BuildingIRN,INT,0,None,0
1,BuildingName,TEXT,0,None,0
2,DistrictIRN,INT,0,None,0
3,DistrictName,TEXT,0,None,0
4,PerformanceIndexScore,TEXT,0,None,0
5,PerformanceIndexPercent,TEXT,0,None,0
6,LetterGradeofPerformanceIndex,TEXT,0,None,0
7,PercentofStudentsBelow,TEXT,0,None,0
8,PercentofStudentsBasic,TEXT,0,None,0
9,PercentofStudentsProficient,TEXT,0,None,0


In [213]:
%%sql
SELECT *
FROM Temp_Achievement_Building_1617
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,PerformanceIndexScore,PerformanceIndexPercent,LetterGradeofPerformanceIndex,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,SchoolYear
224,Adlai Stevenson School,43786,Cleveland Municipal,49.271,41.1,F,60.4,22.5,10.8,5.4,0.8,0.0,2016-2017


In [37]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Achievement_Building_1617

 * sqlite:///CMSD_db.db
Done.


building,District,Name
119,1,1


### Creating Temp Table for Achievement Building School Year 2017-2018

In [42]:
%%sql
CREATE TEMPORARY TABLE Temp_Achievement_Building_1718 AS
SELECT "BuildingIRN", "BuildingName", "DistrictIRN", "DistrictName", "PerformanceIndexScore2017-18" AS "PerformanceIndexScore", "PerformanceIndexPercent2017-18" AS "PerformanceIndexPercent", "LetterGradeofPerformanceIndex", "PercentofStudentsLimited" AS PercentofStudentsBelow, "PercentofStudentsBasic", "PercentofStudentsProficient", "PercentofStudentsAccelerated", "PercentofStudentsAdvanced", "PercentofStudentsAdvancedPlus", '2017-2018' AS SchoolYear
FROM Achievement_Building_1718
WHERE "DistrictIRN" = '043786';

 * sqlite:///CMSD_db.db
Done.


[]

Since I included some additional columns for consideration while working on further analysis, there is one column that I had to rename: PerformanceofStudentsLimited. In the 2017-2018 Achievement table, this column was originally named "PerformanceofStudentsLimited", which deviates from the naming convention used in the two school years prior. For the sake of continuity, I renamed the 2017-2018 column to fit with the prior years, but this change has been noted. 

In [43]:
%%sql
PRAGMA table_info(Temp_Achievement_Building_1718)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,BuildingIRN,INT,0,None,0
1,BuildingName,TEXT,0,None,0
2,DistrictIRN,INT,0,None,0
3,DistrictName,TEXT,0,None,0
4,PerformanceIndexScore,TEXT,0,None,0
5,PerformanceIndexPercent,TEXT,0,None,0
6,LetterGradeofPerformanceIndex,TEXT,0,None,0
7,PercentofStudentsBelow,TEXT,0,None,0
8,PercentofStudentsBasic,TEXT,0,None,0
9,PercentofStudentsProficient,TEXT,0,None,0


In [44]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Achievement_Building_1718

 * sqlite:///CMSD_db.db
Done.


building,District,Name
123,1,1


### Combining the Achievement Temp Tables into one table

In [45]:
%%sql
CREATE TEMPORARY TABLE Combined_Achievement AS
SELECT * FROM Temp_Achievement_Building_1516
UNION ALL
SELECT * FROM Temp_Achievement_Building_1617
UNION ALL
SELECT * FROM Temp_Achievement_Building_1718;

 * sqlite:///CMSD_db.db
Done.


[]

In [214]:
%%sql
SELECT *
FROM Combined_Achievement
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,PerformanceIndexScore,PerformanceIndexPercent,LetterGradeofPerformanceIndex,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,SchoolYear
224,Adlai Stevenson School,43786,Cleveland Municipal,43.903,36.6,F,69.9,19,7.6,2.5,1,0,2015-2016


In [48]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name, COUNT (DISTINCT SchoolYear) AS Year
FROM Combined_Achievement

 * sqlite:///CMSD_db.db
Done.


building,District,Name,Year
128,1,1,3


# Building Ratings
Now, I'm going to narrow down the Building Ratings data in a similar fashion to the Achievement Ratings. I'll go through each year, validate the data, ensure that the correct columns are included, and then union the temporary tables to form a combined Building Ratings table, which can be used to form the final output file.

### Building Ratings 2015-2016

In [215]:
%%sql
SELECT *
FROM Building_Ratings_1516
WHERE DistrictIRN = '043786'
LIMIT 1

/*Just a basic query to check everything is working as intended before making temporary tables

 * sqlite:///CMSD_db.db
Done.


DistrictIRN,DistrictName,BuildingIRN,BuildingName,County,Region,Address,CityStateZip,Phone,Principal,Enrollment2015-2016,LetterGradeofAchievementComponent,LetterGradeofPercentStandards,GiftedIndicatorMet/NotMetStatus,PercentofStandardsMet,LetterGradeofPerformanceIndex,PerformanceIndexPercent,LetterGradeofAMO,AMOPoints,LetterGradeofK3Literacy,K3LiteracyPercent,LetterGradeofProgressComponent,LetterGradeofOverallValueAdded,OverallValueAddedGainIndex,LetterGradeofGiftedValueAdded,GiftedValueAddedGainIndex,LetterGradeofStudentswithDisabilitiesValueAdded,StudentswithDisabilitiesValueAddedGainIndex,LetterGradeofLowest20%ValueAdded,Lowest20%ValueAddedGainIndex,LetterGradeofHighMobilityValueAdded,HighMobilityValueAddedGainIndex,LetterGradeofGradRateComponent,LetterGradeof4YearGradRate2015,4YearGradRate2015,LetterGradeof5YearGradRate2014,5YearGradRate2014,LetterGradeofPreparedforSuccessComponent,PercentofPreparedforSuccessComponent,AttendanceRate2015-2016,AttendanceRate2014-2015,AttendanceRate2013-2014,ChronicAbsenteeismPercent2015-2016,Watermark
43786,Cleveland Municipal City,224,Adlai Stevenson School,Cuyahoga,Region 3,18300 Woda Avenue,"Cleveland, OH, 44122-6441",(216) 482-2950,Christopher T. Wyland,430,F,F,NC,0.0,F,36.6,F,0.0,F,15.8,F,F,-6.6,NR,NC,F,-3.5,F,-4.4,F,-6.1,NR,NR,NC,NR,NC,NR,NC,94.0,92.7,93,17.4,None


In [136]:
%%sql
CREATE TEMPORARY TABLE Temp_Building_Ratings_1516 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "Address", "CityStateZip", "Enrollment2015-2016" AS "Enrollment", "LetterGradeofAchievementComponent", "LetterGradeofPercentStandards", "PercentofStandardsMet", "LetterGradeofPerformanceIndex", "PerformanceIndexPercent", '2015-2016' AS SchoolYear
FROM Building_Ratings_1516
WHERE DistrictIRN = '043786'

 * sqlite:///CMSD_db.db
Done.


[]

In [137]:
%%sql
PRAGMA table_info(Temp_Building_Ratings_1516)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,TEXT,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


In [138]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Building_Ratings_1516

 * sqlite:///CMSD_db.db
Done.


building,District,Name
117,1,1


First potential issue has arisen. Based on the entries remaining in my Temp Tables for 15-16, there are more schools present in the Building Ratings Table than there are in the Achievement Table. The difference is relatively small in absolute terms - 16 entries - however, this is significant.

### Building Ratings 2016-2017

In [171]:
%%sql
DROP TABLE Temp_Building_Ratings_1617

 * sqlite:///CMSD_db.db
Done.


[]

In [172]:
%%sql
CREATE TEMPORARY TABLE Temp_Building_Ratings_1617 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "Address", "CityStateZip", "Enrollment2016-2017" AS "Enrollment", "LetterGradeofAchievementComponent", "LetterGradeofPercentStandards", "PercentofStandardsMet", "LetterGradeofPerformanceIndex", "PerformanceIndexPercent", '2016-2017' AS SchoolYear
FROM Building_Ratings_1617
WHERE DistrictIRN = '043786'

 * sqlite:///CMSD_db.db
Done.


[]

In [173]:
%%sql
PRAGMA table_info(Temp_Building_Ratings_1617)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,TEXT,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


In [174]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Building_Ratings_1617

 * sqlite:///CMSD_db.db
Done.


building,District,Name
116,1,1


In [216]:
%%sql
SELECT *
FROM Temp_Building_Ratings_1617
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


DistrictIRN,DistrictName,BuildingIRN,Address,CityStateZip,Enrollment,LetterGradeofAchievementComponent,LetterGradeofPercentStandards,PercentofStandardsMet,LetterGradeofPerformanceIndex,PerformanceIndexPercent,SchoolYear
43786,Cleveland Municipal City,224,18300 Woda Avenue,"Cleveland, OH, 44122-6441",445,F,F,0.0,F,41.1,2016-2017


Similar issue to 2015-2016, however, 2016-2017 is missing 3 buildings. Based on the number of missing entries for 2017-2018, I think I'll just remove these schools and make note of the issue in the final report.

### Building Ratings 2017-2018

In [217]:
%%sql
SELECT *
FROM Building_Overview_1718
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


DistrictIRN,DistrictName,BuildingIRN,BuildingName,County,Region,Address,CityStateZip,Phone,Principal,Enrollment2017-2018,OverallGrade,AchievementComponentGrade,IndicatorsMetGrade,GiftedIndicatorMet/NotMetStatus,PercentofIndicatorsMet,PerformanceIndexGrade,PerformanceIndexPercent,GapClosingGrade,GapClosingPoints,ImprovingAt-RiskK3ReadersGrade,ImprovingAt-RiskK3ReadersPercent,ProgressComponentGrade,OverallSubgroupValue-AddedGrade,OverallSubgroupValue-AddedGainIndex,GiftedSubgroupValue-AddedGrade,GiftedSubgroupValue-AddedGainIndex,SWDSubgroupValue-AddedGrade,SWDSubgroupValue-AddedGainIndex,Lowest20%SubgroupValue-AddedGrade,Lowest20%SubgroupValue-AddedGainIndex,GradRateComponentGrade,LetterGradeof4YearGradRate2017,4YearGradRate2017,LetterGradeof5YearGradRate2016,5YearGradRate2016,PreparedforSuccessComponentGrade,PreparedforSuccessComponentPercent,AttendanceRate2017-2018,AttendanceRate2016-2017,AttendanceRate2015-2016,ChronicAbsenteeismPercent2017-2018,Watermark
43786,Cleveland Municipal,224,Adlai Stevenson School,Cuyahoga,Region 3,18300 Woda Avenue,"Cleveland, OH, 44122-6441",(216) 838-5300,Christopher T. Wyland,443,D,F,F,NR,0.0,F,40.1,D,62.5,C,37.5,D,F,-8.39,NR,NC,C,-1,F,-5.7,NR,NR,NC,NR,NC,NR,NC,93.5,94.5,94.0,17.6,None


In [149]:
%%sql
CREATE TEMPORARY TABLE Temp_Building_Ratings_1718 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "Address", "CityStateZip", "Enrollment2017-2018" AS "Enrollment", "AchievementComponentGrade" AS "LetterGradeofAchievementComponent", "IndicatorsMetGrade" AS "LetterGradeofPercentStandards", "PercentofIndicatorsMet" AS "PercentofStandardsMet", "PerformanceIndexGrade" AS "LetterGradeofPerformanceIndex", "PerformanceIndexPercent", '2017-2018' AS SchoolYear
FROM Building_Overview_1718
WHERE DistrictIRN = '043786'
/* I might drop the IndicatorsMetGrade column. While this appears to be the same as the LetterGradeofPercentStandards from the previous two years, I can't absolutely
confirm this is the case, and this column isn't of necessary importance. However, to give myself some
flexibility, I'll keep it for now, and then drop it from the unioned table later on.

 * sqlite:///CMSD_db.db
Done.


[]

In [150]:
%%sql
PRAGMA table_info(Temp_Building_Ratings_1718)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,REAL,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


In [151]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Temp_Building_Ratings_1718

 * sqlite:///CMSD_db.db
Done.


building,District,Name
123,1,1


After checking the data types with my PRAGMA query earlier, I can see that PercentofStandardsMet is cast as REAL and PeformanceIndexPercent is cast as TEXT - neither of which are consistent with my other temp tables, so I will recast them to the correct data types now. 

Following this step, the PRAGMA results will show PercentofStandardsMet as TEXT (although it could be a REAL...), and PerformanceIndexPercent as REAL. Once I have finished forming the Union Table, then I can recast them if necessary in the final output file.

In [218]:
%%sql
SELECT CAST(PercentofStandardsMet AS TEXT) AS PercentofStandardsMet, CAST(PerformanceIndexPercent AS REAL) AS PerformanceIndexPercent
FROM Temp_Building_Ratings_1718
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


PercentofStandardsMet,PerformanceIndexPercent
0.0,40.1


In [153]:
%%sql
PRAGMA table_info(Temp_Building_Ratings_1718)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,REAL,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


### Combining Building Ratings into one table

In [175]:
%%sql
DROP TABLE Combined_Building_Ratings

 * sqlite:///CMSD_db.db
Done.


[]

In [176]:
%%sql
CREATE TEMPORARY TABLE Combined_Building_Ratings AS
SELECT * FROM Temp_Building_Ratings_1516
UNION ALL
SELECT * FROM Temp_Building_Ratings_1617
UNION ALL
SELECT * FROM Temp_Building_Ratings_1718;

 * sqlite:///CMSD_db.db
Done.


[]

In [177]:
%%sql
PRAGMA table_info(Combined_Building_Ratings)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


Due to SQLite's type flexibilty, I have two columns that are not cast correctly. So, I'm going to recast them by creating a new table, forming the columns, and then repopulating the data.

In [178]:
%%sql
CREATE TABLE Combined_Building_Ratings_cast (
    DistrictIRN INT,
    DistrictName TEXT,
    BuildingIRN INT,
    Address TEXT,
    CityStateZip TEXT,
    Enrollment INT,
    LetterGradeofAchievementComponent TEXT,
    LetterGradeofPercentStandards TEXT,
    PercentofStandardsMet TEXT,
    LetterGradeofPerformanceIndex TEXT,
    PerformanceIndexPercent REAL,
    SchoolYear TEXT
);

 * sqlite:///CMSD_db.db
Done.


[]

In [179]:
%%sql
INSERT INTO Combined_Building_Ratings_cast
SELECT CAST(DistrictIRN AS INT)
, CAST(DistrictName AS TEXT)
, CAST(BuildingIRN AS INT)
, CAST(Address AS TEXT)
, CAST(CityStateZip AS TEXT)
, CAST(Enrollment AS INT)
, CAST(LetterGradeofAchievementComponent AS TEXT)
, CAST(LetterGradeofPercentStandards AS TEXT)
, CAST(PercentofStandardsMet AS TEXT)
, CAST(LetterGradeofPerformanceIndex AS TEXT)
, CAST(PerformanceIndexPercent AS REAL)
, CAST(SchoolYear AS TEXT)
FROM Combined_Building_Ratings;

 * sqlite:///CMSD_db.db
356 rows affected.


[]

In [180]:
%%sql
DROP TABLE Combined_Building_Ratings;

 * sqlite:///CMSD_db.db
Done.


[]

In [181]:
%%sql
ALTER TABLE Combined_Building_Ratings_cast RENAME TO Combined_Building_Ratings;

 * sqlite:///CMSD_db.db
Done.


[]

In [182]:
%%sql
PRAGMA table_info(Combined_Building_Ratings)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,Address,TEXT,0,None,0
4,CityStateZip,TEXT,0,None,0
5,Enrollment,INT,0,None,0
6,LetterGradeofAchievementComponent,TEXT,0,None,0
7,LetterGradeofPercentStandards,TEXT,0,None,0
8,PercentofStandardsMet,TEXT,0,None,0
9,LetterGradeofPerformanceIndex,TEXT,0,None,0


In [162]:
%%sql
SELECT COUNT (DISTINCT BuildingIRN) AS building, COUNT (DISTINCT DistrictIRN) AS District, COUNT (DISTINCT DistrictName) AS Name
FROM Combined_Building_Ratings

 * sqlite:///CMSD_db.db
Done.


building,District,Name
128,1,2


In [183]:
%%sql
SELECT DISTINCT DistrictName, COUNT(*) AS Count
FROM Combined_Building_Ratings
GROUP BY DistrictName;

 * sqlite:///CMSD_db.db
Done.


DistrictName,Count
Cleveland Municipal,123
Cleveland Municipal City,233


In [184]:
%%sql
SELECT DISTINCT DistrictName, COUNT(*) AS Count
FROM Temp_Building_Ratings_1516
GROUP BY DistrictName;

 * sqlite:///CMSD_db.db
Done.


DistrictName,Count
Cleveland Municipal City,117


In [185]:
%%sql
SELECT DISTINCT DistrictName, COUNT(*) AS Count
FROM Temp_Building_Ratings_1617
GROUP BY DistrictName;

 * sqlite:///CMSD_db.db
Done.


DistrictName,Count
Cleveland Municipal City,116


In [186]:
%%sql
SELECT DISTINCT DistrictName, COUNT(*) AS Count
FROM Temp_Building_Ratings_1718
GROUP BY DistrictName;

 * sqlite:///CMSD_db.db
Done.


DistrictName,Count
Cleveland Municipal,123


In [187]:
%%sql
UPDATE Combined_Building_Ratings
SET DistrictName = 'Cleveland Municipal City'
WHERE DistrictName = 'Cleveland Municipal';

 * sqlite:///CMSD_db.db
123 rows affected.


[]

In [219]:
%%sql
SELECT *
FROM Combined_Building_Ratings
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


DistrictIRN,DistrictName,BuildingIRN,Address,CityStateZip,Enrollment,LetterGradeofAchievementComponent,LetterGradeofPercentStandards,PercentofStandardsMet,LetterGradeofPerformanceIndex,PerformanceIndexPercent,SchoolYear
43786,Cleveland Municipal City,224,18300 Woda Avenue,"Cleveland, OH, 44122-6441",430,F,F,0.0,F,36.6,2015-2016


All done with the Building Ratings! All of my columns look good, the data types are correct, and the table looks good. The only potential issue is the difference in records between this table and my Achievement Table, but we'll cross that bridge after the next section.

# Overall Value Added Grade

Judging by the columns in Overall Grade, I am looking at only taking a few columns from this data. We'll focus on the required columns, and then also take Overall Composite (although I must say, I don't really know what this is right now - but it's numeric and fluctuates, so maybe it's useful). Aside from that, the gifted, composite, and percentile values are likely outside the scope of my analysis.

In [220]:
%%sql
SELECT *
FROM VA_org_1516
WHERE DistrictIRN = '043786'
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


DistrictIRN,DistrictName,BuildingIRN,BuildingName,County,Region,OverallValueAddedGrade,OverallComposite,GiftedValueAddedGrade,GiftedComposite,StudentswithDisabilitiesValueAddedGrade,StudentswithDisabilitiescomposite,Lowest20%ValueAddedGrade,Lowest20%ValueAddedComposite,HighMobilityValueAddedGrade,HighMobilityComposite,Watermark
43786,Cleveland Municipal City,224,Adlai Stevenson School,Cuyahoga,Region 3,F,-6.56,NR,NC,F,-3.48,F,-4.40,F,-6.09,None


In [94]:
%%sql
PRAGMA table_info(VA_org_1516)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INTEGER,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INTEGER,0,None,0
3,BuildingName,TEXT,0,None,0
4,County,TEXT,0,None,0
5,Region,TEXT,0,None,0
6,OverallValueAddedGrade,TEXT,0,None,0
7,OverallComposite,REAL,0,None,0
8,GiftedValueAddedGrade,TEXT,0,None,0
9,GiftedComposite,TEXT,0,None,0


In [103]:
%%sql
CREATE TEMPORARY TABLE Temp_VA_1516 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "OverallValueAddedGrade", "OverallComposite", '2015-2016' AS SchoolYear
FROM VA_org_1516
WHERE DistrictIRN = '043786'

 * sqlite:///CMSD_db.db
Done.


[]

In [104]:
%%sql
PRAGMA table_info(Temp_VA_1516)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,OverallValueAddedGrade,TEXT,0,None,0
4,OverallComposite,REAL,0,None,0
5,SchoolYear,,0,None,0


In [112]:
%%sql
CREATE TEMPORARY TABLE Temp_VA_1617 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "OverallValueAddedGrade", "OverallComposite", '2016-2017' AS SchoolYear
FROM VA_org_1617
WHERE DistrictIRN = '043786'

 * sqlite:///CMSD_db.db
Done.


[]

In [113]:
%%sql
PRAGMA table_info(Temp_VA_1617)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,OverallValueAddedGrade,TEXT,0,None,0
4,OverallComposite,REAL,0,None,0
5,SchoolYear,,0,None,0


In [115]:
%%sql
CREATE TEMPORARY TABLE Temp_VA_1718 AS
SELECT "DistrictIRN", "DistrictName", "BuildingIRN", "OverallValueAddedGrade", "OverallComposite", '2017-2018' AS SchoolYear
FROM VA_ORG_1718
WHERE DistrictIRN = '043786'

 * sqlite:///CMSD_db.db
Done.


[]

In [116]:
%%sql
PRAGMA table_info(Temp_VA_1718)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,OverallValueAddedGrade,TEXT,0,None,0
4,OverallComposite,REAL,0,None,0
5,SchoolYear,,0,None,0


In [117]:
%%sql
CREATE TEMPORARY TABLE Combined_VA AS
SELECT * FROM Temp_VA_1516
UNION ALL
SELECT * FROM Temp_VA_1617
UNION ALL
SELECT * FROM Temp_VA_1718;

 * sqlite:///CMSD_db.db
Done.


[]

In [118]:
%%sql
PRAGMA table_info(Combined_VA)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,DistrictIRN,INT,0,None,0
1,DistrictName,TEXT,0,None,0
2,BuildingIRN,INT,0,None,0
3,OverallValueAddedGrade,TEXT,0,None,0
4,OverallComposite,REAL,0,None,0
5,SchoolYear,,0,None,0


# Final Output File
At this point, I have 3 unioned tables which contain all of the data from each respective category - Achievement Building, Building Rating, and Value Added. The only missing piece currently is that SchoolYear is not defined as TEXT in Combined_Achievement or Combined_VA - however, I can address this once the tables are joined.

In [190]:
%%sql
CREATE TABLE Final_Output_Table AS
SELECT 
    CA.BuildingIRN, 
    CA.BuildingName, 
    CA.DistrictIRN, 
    CA.DistrictName, 
    CA.PerformanceIndexScore, 
    CA.PerformanceIndexPercent,
    CA.LetterGradeofPerformanceIndex,
    CA.PercentofStudentsBelow,
    CA.PercentofStudentsBasic,
    CA.PercentofStudentsProficient,
    CA.PercentofStudentsAccelerated,
    CA.PercentofStudentsAdvanced,
    CA.PercentofStudentsAdvancedPlus,
    CBR.Address,
    CBR.CityStateZip,
    CBR.Enrollment,
    CBR.LetterGradeofAchievementComponent, 
    CVA.OverallValueAddedGrade, 
    CVA.OverallComposite,
    CA.SchoolYear
FROM 
    Combined_Achievement CA
INNER JOIN 
    Combined_Building_Ratings CBR 
    ON CA.BuildingIRN = CBR.BuildingIRN AND CA.SchoolYear = CBR.SchoolYear
INNER JOIN 
    Combined_VA CVA 
    ON CA.BuildingIRN = CVA.BuildingIRN AND CA.SchoolYear = CVA.SchoolYear


 * sqlite:///CMSD_db.db
Done.


[]

In [189]:
%%sql
DROP TABLE Final_Output_Table

 * sqlite:///CMSD_db.db
Done.


[]

In [191]:
%%sql
PRAGMA table_info(Final_Output_Table)

 * sqlite:///CMSD_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,BuildingIRN,INT,0,None,0
1,BuildingName,TEXT,0,None,0
2,DistrictIRN,INT,0,None,0
3,DistrictName,TEXT,0,None,0
4,PerformanceIndexScore,TEXT,0,None,0
5,PerformanceIndexPercent,TEXT,0,None,0
6,LetterGradeofPerformanceIndex,TEXT,0,None,0
7,PercentofStudentsBelow,TEXT,0,None,0
8,PercentofStudentsBasic,TEXT,0,None,0
9,PercentofStudentsProficient,TEXT,0,None,0


In [221]:
%%sql
SELECT *
FROM Final_Output_Table
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,PerformanceIndexScore,PerformanceIndexPercent,LetterGradeofPerformanceIndex,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,Address,CityStateZip,Enrollment,LetterGradeofAchievementComponent,OverallValueAddedGrade,OverallComposite,SchoolYear
224,Adlai Stevenson School,43786,Cleveland Municipal,43.903,36.6,F,69.9,19,7.6,2.5,1,0,18300 Woda Avenue,"Cleveland, OH, 44122-6441",430,F,F,-6.56,2015-2016


### Checking the final output table and data validation

In [193]:
%%sql
SELECT COUNT(*) AS TotalRows FROM Final_Output_Table;

 * sqlite:///CMSD_db.db
Done.


TotalRows
340


In [194]:
%%sql
SELECT COUNT(DISTINCT DistrictName) AS UniqueDistrictNames, COUNT(DISTINCT SchoolYear) AS UniqueSchoolYears FROM Final_Output_Table;

 * sqlite:///CMSD_db.db
Done.


UniqueDistrictNames,UniqueSchoolYears
1,3


In [129]:
%%sql
SELECT COUNT(*) AS NullCount FROM Final_Output_Table WHERE BuildingIRN IS NULL;

 * sqlite:///CMSD_db.db
Done.


NullCount
0


In [130]:
%%sql
SELECT BuildingIRN, BuildingName, COUNT(*) AS NumOccurrences 
FROM Final_Output_Table 
GROUP BY BuildingIRN, BuildingName 
HAVING COUNT(*) > 3; -- or change to "= 3" to check if there are exactly 3 occurrences

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,NumOccurrences


In [209]:
%%sql
SELECT *
FROM Final_Output_Table 
LIMIT 1

 * sqlite:///CMSD_db.db
Done.


BuildingIRN,BuildingName,DistrictIRN,DistrictName,PerformanceIndexScore,PerformanceIndexPercent,LetterGradeofPerformanceIndex,PercentofStudentsBelow,PercentofStudentsBasic,PercentofStudentsProficient,PercentofStudentsAccelerated,PercentofStudentsAdvanced,PercentofStudentsAdvancedPlus,Address,CityStateZip,Enrollment,LetterGradeofAchievementComponent,OverallValueAddedGrade,OverallComposite,SchoolYear
224,Adlai Stevenson School,43786,Cleveland Municipal,43.903,36.6,F,69.9,19,7.6,2.5,1,0,18300 Woda Avenue,"Cleveland, OH, 44122-6441",430,F,F,-6.56,2015-2016


In order to save the final output table, the following command needs to be executed in terminal (or command prompt, although I did this project using Mac).

In [ ]:
sqlite3 -header -csv 'CMSD_db.db' "SELECT * FROM Final_Output_Table;" > final_output.csv

## Python Code for creating my pivot table
As I was working through my analysis, I realized I would like to have some data for PIP and VA by year, as well the changes each experienced year to year. I considered doing this in Excel, but opted for Python for repeatability. The general process is taking the data from each year and placing it into columns based on each year - so I ended up with columns for PIP in each school year, Composite Score for each year, and VA Grade for each year.

In [224]:
# Load my csv as a Pandas dataframe for manipulation
import pandas as pd
data = '/Users/camerongomez/Desktop/CMSD Downloads/final_output.csv'
cmsd_data = pd.read_csv(data)

In [225]:
# Creating the year columns
cmsd_data['Year'] = cmsd_data['SchoolYear'].str.split('-').str[0]

cmsd_data['Year'].unique()

array(['2015', '2016', '2017'], dtype=object)

In [226]:
# Filtering for schools for the subset
first_school = cmsd_data.iloc[0]['BuildingIRN']
subset_data = cmsd_data[cmsd_data['BuildingIRN'] == first_school]

# pivot column naming
pivot_columns = ['PerformanceIndexPercent', 'LetterGradeofPerformanceIndex', 'OverallComposite', 'OverallValueAddedGrade']

# Create a dictionary for renaming columns after pivoting
rename_dict = {
    'PerformanceIndexPercent': 'PIP',
    'LetterGradeofPerformanceIndex': 'Letter_Grade',
    'OverallComposite': 'Comp',
    'OverallValueAddedGrade': 'VA'
}

pivoted_data = subset_data.pivot(index = 'BuildingIRN', columns = 'Year', values = pivot_columns)

# Rename the columns
pivoted_data.columns = ['_'.join([rename_dict[col[0]], col[1]]) for col in pivoted_data.columns]

# Reset index to make 'BuildingIRN' a column again - I made a series of mistakes after this in my first attempt.
pivoted_data.reset_index(inplace=True)

pivoted_data

,BuildingIRN,PIP_2015,PIP_2016,PIP_2017,Letter_Grade_2015,Letter_Grade_2016,Letter_Grade_2017,Comp_2015,Comp_2016,Comp_2017,VA_2015,VA_2016,VA_2017
0,224,36.6,41.1,40.1,F,F,F,-6.56,-8.05,-8.39,F,F,F


I had some issues with this section due to a naming error with the dataframe index and needed to get some help. It works as intended, but took me a little while.

In [235]:
# Pivot the data for all schools to have columns for each year's PIP, Letter Grade, Overall Composite, and VA Grade.
pivoted_data_full = cmsd_data.pivot_table(index=['BuildingIRN', 'BuildingName', 'DistrictIRN'], 
                                           columns='Year', 
                                           values=pivot_columns,
                                           aggfunc='first')  # using 'first' to keep the first occurrence in case of duplicates

# Flatten columns
pivoted_data_full.columns = ['_'.join(col).strip() for col in pivoted_data_full.columns.values]

# Correct the column names
corrected_columns = {}
for col in pivoted_data_full.columns:
    # Correct the columns...
    if '_' in col and col.split('_')[0] in rename_dict:
        main_part, year = col.rsplit('_', 1)
        corrected_columns[col] = f"{rename_dict.get(main_part, main_part)}_{year}"  # Use the rename dict to get the correct main part

# Rename the columns
pivoted_data_full.rename(columns=corrected_columns, inplace=True)

# Convert the PIP and Comp columns to numeric type for calculations (they were objects)
for col in pivoted_data_full.columns:
    if "PIP_" in col or "Comp_" in col:
        pivoted_data_full[col] = pd.to_numeric(pivoted_data_full[col], errors='coerce')

# Confirm column names
pivoted_data_full.columns.tolist()

['Letter_Grade_2015',
 'Letter_Grade_2016',
 'Letter_Grade_2017',
 'Comp_2015',
 'Comp_2016',
 'Comp_2017',
 'VA_2015',
 'VA_2016',
 'VA_2017',
 'PIP_2015',
 'PIP_2016',
 'PIP_2017']

In [236]:
# Reset the index
pivoted_data_full.reset_index(inplace=True)

# Check df
pivoted_data_full.head(3)

,BuildingIRN,BuildingName,DistrictIRN,Letter_Grade_2015,Letter_Grade_2016,Letter_Grade_2017,Comp_2015,Comp_2016,Comp_2017,VA_2015,VA_2016,VA_2017,PIP_2015,PIP_2016,PIP_2017
0,224,Adlai Stevenson School,43786,F,F,F,-6.56,-8.05,-8.39,F,F,F,36.6,41.1,40.1
1,318,Menlo Park Academy,43786,NaN,B,A,NaN,-10.14,-4.99,NaN,F,F,NaN,88.7,90.7
2,489,Almira,43786,F,F,F,-6.16,-9.32,-7.71,F,F,F,38.7,39.0,41.6


In [ ]:
# Function to calculate year-to-year changes with handling for missing data - I needed help with this section
def calculate_yearly_changes(row, column_prefix, years):
    changes = {}
    for i in range(len(years) - 1):
        year1, year2 = years[i], years[i + 1]
        col1, col2 = f"{column_prefix}_{year1}", f"{column_prefix}_{year2}"
        change_col = f"{column_prefix}_Change_{year1[-2:]}-{year2[-2:]}"  # e.g., PIP_Change_16-17

        # Calculate the change if both years have data, else set as NaN
        if pd.notnull(row[col1]) and pd.notnull(row[col2]):
            changes[change_col] = row[col2] - row[col1]
        else:
            changes[change_col] = float('nan')  # Set as NaN to indicate missing data
    return pd.Series(changes)
# Calculate the year-to-year changes for PIP and Overall Composite
years = ['2015', '2016', '2017']
pivoted_data_full = pivoted_data_full.join(pivoted_data_full.apply(lambda row: calculate_yearly_changes(row, 'PIP', years), axis=1))
pivoted_data_full = pivoted_data_full.join(pivoted_data_full.apply(lambda row: calculate_yearly_changes(row, 'Comp', years), axis=1))

# Display the first few rows of the dataset
pivoted_data_full.head(3)

In [ ]:
# Calculate the year-to-year changes for PIP and Overall Composite
years = ['2015', '2016', '2017']
pivoted_data_full = pivoted_data_full.join(pivoted_data_full.apply(lambda row: calculate_yearly_changes(row, 'PIP', years), axis=1))
pivoted_data_full = pivoted_data_full.join(pivoted_data_full.apply(lambda row: calculate_yearly_changes(row, 'Comp', years), axis=1))

# Display the first few rows of the dataset to verify the new columns and calculations
pivoted_data_full.head(3)

In [239]:
# Saving the transformed dataset
output_file_path = '/Users/camerongomez/Desktop/CMSD Downloads/pivot_CMSD.csv'

# DataFrame to a CSV file
pivoted_data_full.to_csv(output_file_path, index=False)

output_file_path


'/Users/camerongomez/Desktop/CMSD Downloads/final.csv'